In [28]:
pip install transformers datasets evaluate accelerate

In [3]:
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[{'label': 'ENTAILMENT', 'score': 0.9883741140365601}]

In [4]:
from datasets import load_dataset

imdb = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
# Sample only 500 rows for each split
imdb = {
    "train": imdb["train"].shuffle(seed=42).select(range(500)),
    "test": imdb["test"].shuffle(seed=42).select(range(500)),
    "unsupervised": imdb["unsupervised"].shuffle(seed=42).select(range(500))
}

In [6]:
for split_name, split_data in imdb.items():
    print(f"{split_name} shape: ({split_data.num_rows}, {len(split_data.features)})")

train shape: (500, 2)
test shape: (500, 2)
unsupervised shape: (500, 2)


In [7]:
from datasets import DatasetDict
imdb = DatasetDict(imdb)

In [8]:
imdb["test"][0]

{'text': "<br /><br />When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could go wrong?<br /><br />Very quickly, however, I realized that this story was about A Thousand Other Things besides just Acres. I started crying and couldn't stop until long after the movie ended. Thank you Jane, Laura and Jocelyn, for bringing us such a wonderfully subtle and compassionate movie! Thank you cast, for being involved and portraying the characters with such depth and gentleness!<br /><br />I recognized the Angry sister; the Runaway sister and the sister in Denial. I recognized the Abusive Husband and why he was there and then the Father, oh oh the Father... all superbly played. I also recognized myself and this movie was an eye-opener, a relief, a chance to face my OWN truth and finally doing something about it. I truly hope A Thousand Acres has had the same effect on some others out there.<br /><b

In [9]:
from transformers import AutoTokenizer  # for personal using , finetuning , custom task
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased") #200

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)  # fixed size Example 1024

In [11]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [12]:
tokenized_imdb["train"][0]   #id for every word

{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...',
 'label': 1,
 'input_ids': [101,
  2045,
  2003,
  2053,
  7189,
  2012,
  2035,
  2090,
  3481,
  3771,
  1998,
  6337,
  2099,
  2021,
  1996,
  2755,
  2008,
  2119,
  2024,
  2610,
  2186,
  2055,
  6355,
  6997,
  1012,
  

In [13]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
import evaluate
accuracy = evaluate.load("accuracy")

In [15]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [16]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [17]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
from huggingface_hub import notebook_login
notebook_login()

In [30]:
training_args = TrainingArguments(
    output_dir="SentimentAnalysis",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-30-56224ef383cb>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.318746,0.862000
2,No log,0.302648,0.872000


TrainOutput(global_step=64, training_loss=0.24079705774784088, metrics={'train_runtime': 4494.7963, 'train_samples_per_second': 0.222, 'train_steps_per_second': 0.014, 'total_flos': 131345565373632.0, 'train_loss': 0.24079705774784088, 'epoch': 2.0})

In [31]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/Menna-Khaled/SentimentAnalysis/commit/20af67332409f6a64072835bb27a463141008003', commit_message='End of training', commit_description='', oid='20af67332409f6a64072835bb27a463141008003', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Menna-Khaled/SentimentAnalysis', endpoint='https://huggingface.co', repo_type='model', repo_id='Menna-Khaled/SentimentAnalysis'), pr_revision=None, pr_num=None)

In [32]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="Menna-Khaled/SentimentAnalysis")
classifier(text)

config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.8874754905700684}]

In [33]:
text = "that was great"

from transformers import pipeline

classifier = pipeline("sentiment-analysis",  model="Menna-Khaled/SentimentAnalysis")
classifier(text)

[{'label': 'POSITIVE', 'score': 0.707796573638916}]

In [34]:
text = "very bad opinion"

from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="Menna-Khaled/SentimentAnalysis")
classifier(text)

[{'label': 'NEGATIVE', 'score': 0.687042772769928}]